In [1]:
# qc rough alignment in downsampled stack
#   differs from render-modules
#     point-match derived residuals including angular residuals


In [2]:
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
import concurrent.futures
import csv
import json
import urllib
import uuid

import numpy
import renderapi
import shapely.geometry
import shapely.ops
import shapely.affinity

In [28]:
render_connect = {
   "owner": "TEM",
    "project": "409828_L1",
    "port": 8888,
    "host": "em-131db2.corp.alleninstitute.org",
    "client_scripts": ""
}
r = renderapi.connect(**render_connect)

In [5]:
# map_fn = "/allen/programs/celltypes/workgroups/em-connectomics/russelt/20200428_grid_idx_z_idx_mapping.tsv"
# with open(map_fn, "r") as f:
#     z_mapping = {int(k): int(v) for k, v in csv.reader(f, delimiter="\t")}
# inv_z_mapping = {v: k for k, v in z_mapping.items()}

map_fn = "/allen/programs/celltypes/workgroups/em-connectomics/russelt/20210111_zidx_sectionidx_map.tsv"
with open(map_fn, "r") as f:
    z_to_sid = {int(k): int(v) for k, v in csv.reader(f, delimiter="\t")}
sid_to_z = {v: k for k, v in z_to_sid.items()}

In [29]:
# ra_stack = "20200609_tps25_aligned"
ra_stack = "20210114_inhib_tps25_ra"
montage_stack = "em_2d_montage_solved_py"

ra_pm_collection = "21617_R1_rough_matches"

ds_scale = 0.01

In [30]:
ra_stack = "rough_aligned_tps_remapped"
montage_stack = "20201020_alignment_montages"

ra_pm_collection = "v1dd_409828_L1_rough_matches_remapped"

ds_scale = 0.01

In [7]:
# tosolve_zs_fn = "/allen/programs/celltypes/workgroups/em-connectomics/russelt/20201023_redone_zs.txt"

# with open(tosolve_zs_fn, "r") as f:
#     zs_tosolve = [int(ln.strip()) for ln in f.readlines()]

In [8]:
# g_z_map_fn = "/allen/programs/celltypes/workgroups/em-connectomics/russelt/20201021_v1dd_materialization/20201023_v1dd_redone_grididx_zidx.tsv"

# with open(g_z_map_fn, "w") as f:
#     for z in set(zs_tosolve) & set(ra_zs):
#         f.write(f"{inv_z_mapping[int(z)]}\t{int(z)}\n")



In [31]:
def combine_resolvedtiles(rts_l):
    tforms = {i.transformId: i for l in (rts.transforms for rts in rts_l) for i in l}
    tspecs = list({ts.tileId: ts for l in (rts.tilespecs for rts in rts_l) for ts in l}.values())
    
    return renderapi.resolvedtiles.ResolvedTiles(transformList=tforms, tilespecs=tspecs)

In [32]:
def angle_between_points(pts_a, pts_b, observer_pts=None):
    """get angle between points in NxM arrays with optional observer points
    where N is number of points and M is dimension of points
    """
    if observer_pts is None:
        observer_pts = numpy.zeros_like(pts_a)
    a = pts_a - observer_pts
    b = pts_b - observer_pts
    return numpy.arccos(
        numpy.einsum("ij,ij->i", a, b) /  # vectorized dot product
        numpy.linalg.norm(a, axis=1) /
        numpy.linalg.norm(b, axis=1))

In [33]:
solve_range = (1, 17347)

ra_zs = [z for z in renderapi.stack.get_z_values_for_stack(ra_stack, render=r) if solve_range[0] <= z <= solve_range[-1]]


In [12]:
num_threads = 60
with concurrent.futures.ProcessPoolExecutor(max_workers=num_threads) as e:
    rtfut_to_z = {e.submit(renderapi.resolvedtiles.get_resolved_tiles_from_z, ra_stack, z, render=r): z for z in ra_zs}
    
    z_to_rts = {rtfut_to_z[fut]: fut.result() for fut in concurrent.futures.as_completed(rtfut_to_z)}
    

In [13]:
matchgroups = {ts.layout.sectionId for ts in (i for l in (rts.tilespecs for rts in z_to_rts.values()) for i in l)}

with concurrent.futures.ProcessPoolExecutor(max_workers=num_threads) as e:
    futs = [e.submit(renderapi.pointmatch.get_matches_with_group, ra_pm_collection, g, render=r) for g in matchgroups]
    
    allmatches = [i for l in (fut.result() for fut in concurrent.futures.as_completed(futs)) for i in l]

In [14]:
tileId_to_tileId_match = {(m["pId"], m["qId"]): m for m in allmatches}

In [15]:
combined_resolvedtiles = combine_resolvedtiles(z_to_rts.values())
tileId_to_tilespecs = {ts.tileId: ts for ts in combined_resolvedtiles.tilespecs}

In [16]:
def match_worldcoords_ts(ts, matcharr, sharedTransforms=[]):
    # p_matches = numpy.array(match["matches"]["p"]).T
    # q_matches = numpy.array(match["matches"]["q"]).T
    
    return renderapi.transform.utils.estimate_dstpts(
        ts.tforms, src=matcharr, reference_tforms=sharedTransforms)
    # q_tformed = renderapi.transform.utils.estimate_dstpts(
    #     q_ts.tforms, src=q_matches, reference_tforms=sharedTransforms)
    # return q_tformed - p_tformed

# TODO include weights?
def residuals_from_ts(p_ts, q_ts, match, sharedTransforms=[]):
    p_matches = numpy.array(match["matches"]["p"]).T
    q_matches = numpy.array(match["matches"]["q"]).T
    
    p_tformed = renderapi.transform.utils.estimate_dstpts(
        p_ts.tforms, src=p_matches, reference_tforms=sharedTransforms)
    q_tformed = renderapi.transform.utils.estimate_dstpts(
        q_ts.tforms, src=q_matches, reference_tforms=sharedTransforms)
    return q_tformed - p_tformed


In [17]:
tileId_to_tileId_tformed_pts = {}
for (pId, qId), match in tileId_to_tileId_match.items():
    try:
        p_ts = tileId_to_tilespecs[pId]
        q_ts = tileId_to_tilespecs[qId]
    except KeyError:
        continue
    p_matches = numpy.array(match["matches"]["p"]).T
    q_matches = numpy.array(match["matches"]["q"]).T
    

    tileId_to_tileId_tformed_pts[(pId, qId)] = (
        match_worldcoords_ts(
            p_ts, p_matches, combined_resolvedtiles.transforms),
        match_worldcoords_ts(
            q_ts, q_matches, combined_resolvedtiles.transforms)
    )


In [18]:
# center of mass calculation

# for now, proxying by doing center of source point matches
tileId_to_tileId_residuals = {}
tileId_to_tileId_angular_residuals = {}

for (pId, qId), (p_pts, q_pts) in tileId_to_tileId_tformed_pts.items():
    p_ctr = p_pts.mean(axis=0)

    ang_residuals = angle_between_points(p_pts, q_pts, p_ctr)
    residuals = p_pts - q_pts
    
    tileId_to_tileId_residuals[(pId, qId)] = residuals
    tileId_to_tileId_angular_residuals[(pId, qId)] = ang_residuals

<ipython-input-10-ff4aca974ccd>:9: RuntimeWarning: invalid value encountered in arccos
  return numpy.arccos(


In [19]:
tileId_to_z = {ts.tileId: ts.z for ts in combined_resolvedtiles.tilespecs}

In [20]:
def get_ctrthreshold_filter(arr, threshold):
    return numpy.linalg.norm(arr - arr.mean(axis=0), axis=1) > threshold

In [21]:
depth = 2
bad_pIds, bad_qIds = zip(*[(pId, qId) for (pId, qId), angarr in tileId_to_tileId_angular_residuals.items()
    if abs(tileId_to_z[pId] - tileId_to_z[qId]) < depth
    and numpy.rad2deg(angarr[get_ctrthreshold_filter(
        tileId_to_tileId_tformed_pts[(pId, qId)][0], 35)].max()) >= 5])

In [22]:
len(set(bad_pIds)), len(set(bad_qIds)), len(set(bad_pIds) | set(bad_qIds))

(28, 28, 56)

In [23]:
bad_tiles = set(bad_pIds) | set(bad_qIds)
len(bad_tiles)

56

In [ ]:
for bad_z in sorted({tileId_to_z[tId] for tId in bad_tiles}):
    print(int(bad_z))

In [ ]:
somezs = [241, 464, 690, 760, 984, 1151, 1657, 1897, 3155]
for z in somezs:
    print(inv_z_mapping[z])

In [24]:
# angular residuals
import numpy as np
median_angular_residuals = []
for k in tileId_to_tileId_angular_residuals.items():
    median_angular_residuals.append(np.nanmedian(k[1]))

In [27]:
print(len(median_angular_residuals))
import pandas as pd
print(len(ra_zs))

df = pd.DataFrame(data={"median residual": median_angular_residuals})
df.to_pickle("/allen/programs/celltypes/workgroups/em-connectomics/gayathrim/paper/inhibitory_rough_angular_residuals.pkl")

69042
17305
